In [13]:
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_csv('data_scientist_united_states_job_postings_jobspikr.csv', low_memory = True)

In [16]:
drop_cols = ['crawl_timestamp', 'salary_offered', 'job_board','geo', 'cursor', 'contact_email', 'contact_phone_number', 'html_job_description', 'inferred_city', 'inferred_state', 'inferred_country']
df = df.drop(columns  = drop_cols)


In [20]:
df = df.dropna(axis = 0, how = 'any', subset = None, inplace = False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9992
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              8700 non-null   object
 1   job_title        8700 non-null   object
 2   category         8700 non-null   object
 3   company_name     8700 non-null   object
 4   city             8700 non-null   object
 5   state            8700 non-null   object
 6   country          8700 non-null   object
 7   post_date        8700 non-null   object
 8   job_description  8700 non-null   object
 9   job_type         8700 non-null   object
 10  uniq_id          8700 non-null   object
dtypes: object(11)
memory usage: 815.6+ KB


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              10000 non-null  object
 1   job_title        10000 non-null  object
 2   category         9118 non-null   object
 3   company_name     9999 non-null   object
 4   city             9751 non-null   object
 5   state            9584 non-null   object
 6   country          10000 non-null  object
 7   post_date        10000 non-null  object
 8   job_description  10000 non-null  object
 9   job_type         10000 non-null  object
 10  uniq_id          10000 non-null  object
dtypes: object(11)
memory usage: 859.5+ KB


In [9]:
#change the types of the columns from object 
df['category'] = df['category'].astype('category')
df['company_name'] = df['company_name'].astype('string')
df['city'] = df['city'].astype('string')
df['state'] = df['state'].astype('string')
df['country'] = df['country'].astype('string')
df['job_type'] = df['job_type'].astype('category')

In [ ]:
## updates 

In [12]:
df['job_description'].apply(lambda x : len(x.split())).sum()

5642623

In [11]:
corpus = df['job_description']

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
corpus = df['job_description']
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(corpus)

#Dataframe with cosine similarties
cos_df = pd.DataFrame(cosine_similarity(X_train_counts))

#Dataframe for job advert and similarity with every other advert
i, j = np.indices(cos_df.shape).reshape(2, -1)
cos_values = cos_df.values.reshape(-1)
cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'sim':cos_values})

#Any advert that has a cosine similarity above 0.98 and doesn't equal itself
cos_rem = cos_sim_df[(cos_sim_df['sim']>0.98)&(i!=j)]

#Method to remove duplicates but keep first instance. Trying to drop duplicates on i and j column wouldn't work as the numbers don't duplicate in those
#columns. Setting another column that combines their values ensure that duplicates can be dropped.

cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)

#Drop from original dataframe
df = df[~df.index.isin(drop_rows)]
df = df.reset_index()
df.drop(columns={'index'}, inplace=True)

In [13]:
import nltk 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import re

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

#Function to pre-process the text information

def normalize_document(doc):
    
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A) #re.I (ignore case), re.A (ASCII-only matching)
    doc = doc.lower()
    doc = doc.strip()
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatise document from filtered tokens
    lem_text = [lemmatizer.lemmatize(i) for i in filtered_tokens]
    
    # Remove words that are only one character.
    lem_text = [token for token in lem_text if len(token) > 1]
    
    # Remove numbers, but not words that contain numbers.
    lem_text = [token for token in lem_text if not token.isnumeric()]
    
    doc = ' '.join(lem_text)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus)

In [28]:
from nltk.tokenize import word_tokenize
import gensim.corpora as corpora
nltk.download('punkt')

#Tokenise the corpus
tokenized_corp = [word_tokenize(i) for i in norm_corpus]

# Create Dictionary
id2word = corpora.Dictionary(tokenized_corp)

#Remove words that don't feature 20 times and those that feature in over 50% of documents
id2word.filter_extremes(no_below=20, no_above=0.5)

texts = tokenized_corp

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/wendionwuakpa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [33]:
import gensim
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=9, 
                                       random_state=123,
                                       chunksize=100,
                                       passes=100,
                                       per_word_topics=True)

In [32]:
# num_topics = 4
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"status" + 0.015*"employment" + 0.014*"disability" + 0.012*"veteran" '
  '+ 0.012*"protected" + 0.012*"applicant" + 0.012*"national" + 0.012*"gender" '
  '+ 0.011*"equal" + 0.010*"race"'),
 (1,
  '0.007*"advanced" + 0.007*"modeling" + 0.007*"analytical" + '
  '0.006*"management" + 0.006*"process" + 0.006*"complex" + 0.006*"support" + '
  '0.006*"including" + 0.006*"insight" + 0.005*"technical"'),
 (2,
  '0.011*"position" + 0.010*"research" + 0.009*"required" + 0.008*"health" + '
  '0.008*"information" + 0.007*"must" + 0.007*"employee" + 0.006*"support" + '
  '0.006*"program" + 0.006*"education"'),
 (3,
  '0.014*"product" + 0.009*"company" + 0.008*"customer" + 0.007*"people" + '
  '0.007*"help" + 0.006*"build" + 0.005*"world" + 0.005*"role" + '
  '0.005*"insight" + 0.005*"looking"')]


In [34]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"client" + 0.014*"service" + 0.013*"financial" + 0.011*"company" + '
  '0.009*"risk" + 0.008*"management" + 0.008*"industry" + 0.008*"help" + '
  '0.007*"global" + 0.007*"customer"'),
 (1,
  '0.015*"system" + 0.013*"management" + 0.013*"process" + 0.011*"support" + '
  '0.008*"design" + 0.007*"requirement" + 0.007*"technical" + 0.006*"quality" '
  '+ 0.005*"software" + 0.005*"provide"'),
 (2,
  '0.024*"research" + 0.022*"health" + 0.016*"healthcare" + 0.013*"clinical" + '
  '0.011*"care" + 0.009*"patient" + 0.008*"scientific" + 0.007*"computational" '
  '+ 0.006*"medical" + 0.006*"preferred"'),
 (3,
  '0.020*"product" + 0.010*"people" + 0.009*"customer" + 0.008*"help" + '
  '0.008*"build" + 0.007*"company" + 0.007*"world" + 0.006*"insight" + '
  '0.006*"engineering" + 0.006*"make"'),
 (4,
  '0.012*"modeling" + 0.012*"insight" + 0.012*"advanced" + 0.011*"analytical" '
  '+ 0.009*"predictive" + 0.009*"complex" + 0.008*"quantitative" + '
  '0.007*"decision" + 0.007*"result" 

In [ ]:
## dont need 
l = list(df['company_name'])

# Pass the split_it list to instance of Counter class.
Counter = Counter(l)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(20)
  
print(most_occur)

from collections import Counter
l = list(df['job_description'])
s = ""
for word in l: 
    if(len(word) >= 5):
        s += word

split_it = s.split()
# Pass the split_it list to instance of Counter class.
Counter = Counter(split_it)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(10)
  
print(most_occur)

In [35]:
topics_df1 = pd.DataFrame()
topics_df2 = pd.DataFrame()
topics_df3 = pd.DataFrame()
 
for i, row_list in enumerate(lda_model[corpus]):
    row = row_list[0] if lda_model.per_word_topics else row_list            
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if len(row) >= 3:        
            if j ==0:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==1:
                topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==2:
                topics_df3 = topics_df3.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            else:
                break
        elif len(row) == 2:
            if j ==0:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==1:
                topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
                topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)
        elif len(row) == 1:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
                topics_df2 = topics_df2.append(pd.Series(['-', '-']), ignore_index=True)  
                topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)        
                
            
topics_df1.rename(columns={0:'1st Topic', 1:'1st Topic Contribution'}, inplace=True)
topics_df2.rename(columns={0:'2nd Topic', 1:'2nd Topic Contribution'}, inplace=True)
topics_df3.rename(columns={0:'3rd Topic', 1:'3rd Topic Contribution'}, inplace=True)

topics_comb = pd.concat([topics_df1, topics_df2, topics_df3],  axis=1, sort=False)

#Join topics dataframe to original data
df = pd.concat([df, topics_comb], axis=1, sort=False)

#Combine the top 3 into one column so it's easier to analyse
df['Top 3 Topics'] = df['1st Topic'].apply(lambda x: lda_dict[x].join(" ,") + " ").astype(str) + df['2nd Topic'].apply(lambda x: lda_dict[x].join(" ,") + " ").astype(str) + \
df['3rd Topic'].apply(lambda x: lda_dict[x]).astype(str)

/var/folders/rz/vdqsbcg13dn2ntcv37cds99c0000gn/T/ipykernel_54907/1235325460.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
/var/folders/rz/vdqsbcg13dn2ntcv37cds99c0000gn/T/ipykernel_54907/1235325460.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
/var/folders/rz/vdqsbcg13dn2ntcv37cds99c0000gn/T/ipykernel_54907/1235325460.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topics_df3 = topics_df3.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
/var/folders/rz/vdqsbcg13dn2ntcv37cds99c0000gn/T/ipykernel_54907/12353254

NameError: name 'lda_dict' is not defined

In [52]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

# import spaCY

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim_models

import matplotlib.pyplot as plt
import pandas as pd

In [55]:
#import pyLDAvis.gensim
# import pickle 
# import pyLDAvis
# from pyLDAvis import gensim
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/we

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.124404  0.053342       1        1  19.115175
3      0.061079 -0.095121       2        1  17.624511
1      0.097334  0.071188       3        1  12.821435
7      0.112835 -0.072292       4        1  10.390560
8     -0.087885  0.138950       5        1   9.894565
6     -0.282003  0.058271       6        1   8.957706
0      0.023553 -0.050581       7        1   8.105293
5     -0.123767 -0.191519       8        1   7.102590
2      0.074449  0.087762       9        1   5.988165, topic_info=             Term         Freq        Total Category  logprob  loglift
352        status  4164.000000  4164.000000  Default  30.0000  30.0000
18         client  5040.000000  5040.000000  Default  29.0000  29.0000
461      research  6365.000000  6365.000000  Default  28.0000  28.0000
255    disability  3076.000000  3076.000000  Default  27.0000  27.0000
262    employment  3289.000000  3289.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
41    environment   384.006343  5055.772736   Topic9  -5.4235   0.2378
111      required   389.421546  5737.515800   Topic9  -5.4095   0.1253
474       support   377.956386  5639.735620   Topic9  -5.4393   0.1126
709        member   330.222636  2340.371262   Topic9  -5.5743   0.8571
1134     approach   307.399237  2115.223345   Topic9  -5.6460   0.8866

[735 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
2716      5  0.961560      )
2716      7  0.002081      )
2716      9  0.035382      )
2717      4  0.003430      ,
2717      5  0.936366      ,
...     ...       ...    ...
564       4  0.251045  youll
564       7  0.073779  youll
564       8  0.061400  youll
1372      2  0.965028  youve
1372      7  0.033742  youve

[2476 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 8, 9, 7, 1, 6, 3])

In [44]:
#import pyLDAvis.gensim

ModuleNotFoundError: No module named 'pyLDAvis.gensim'